# Nabewerking Klimaatsommen

### Draai eerst 03_nabewerking_klimaatsommen_prep.ipynb

Zorg dat in de poldermap onder 01. DAMO en HDB een mapje staat met de naam 'peilgebieden'.
In dit mapje mapje moet de shape van de peilgebieden. Deze shape moet 'peilgebieden_Polderclusternaam.shp' heten,
dus bijvoorbeeld 'peilgebieden_Eijerland.shp'. In de kolom 'name' moet de naam van de polder staan.
Dit laatste is van belang wanneer meerdere polders in één poldercluster gezamelijk zijn doorgerekend.
Maak een kopie van je datachecker output: fixeddrainagelevelarea. Dat werkt.


In [ ]:
from notebook_setup import setup_notebook
notebook_data = setup_notebook()

import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import sys
import importlib.resources as pkg_resources  # Load resource from package
import ipywidgets as widgets

from hhnk_threedi_tools import Folders
import hhnk_threedi_tools as htt
import hhnk_research_tools as hrt

# import hhnk_threedi_tools.core.climate_scenarios as hrt_climate
import hhnk_threedi_tools.core.climate_scenarios.maskerkaart as maskerkaart
import hhnk_threedi_tools.core.climate_scenarios.ruimtekaart as ruimtekaart
from hhnk_threedi_tools.core.climate_scenarios.interpolate_rasters import (
    main_interpolate_rasters,
)
import hhnk_threedi_tools.core.climate_scenarios.schadekaart as schadekaart
import hhnk_threedi_tools.core.climate_scenarios.peilgebieden as peilgebieden
from hhnk_threedi_tools.core.climate_scenarios.schadekaart_peilgebieden import maak_schade_polygon
from hhnk_threedi_tools.core.climate_scenarios.maskerkaart_raster import rasterize_maskerkaart
from hhnk_threedi_tools.core.climate_scenarios.klimaatsommen_prep import KlimaatsommenPrep

# Folders inladen
folder = Folders(notebook_data['polder_folder'])

# Of handmatig;
# folder=Folders(r"E:\02.modellen\model_test_v2")


## Selectie neerslagzone en dem

In [ ]:
def item_layout(width="95%", grid_area="", **kwargs):
    return widgets.Layout(
        width=width, grid_area=grid_area, **kwargs
    )  # override the default width of the button to 'auto' to let the button grow


output_folder_options = [""] + folder.threedi_results.batch.content

# output_folder_label = widgets.Label('Selecteer output folder:', layout=item_layout(grid_area='output_folder_label'))
output_folder_box = widgets.Select(
    options=output_folder_options,
    rows=len(output_folder_options),
    disabled=False,
    layout=item_layout(grid_area="output_folder_box"),
)


# Set dem path
folder.model.set_modelsplitter_paths()
dem = hrt.Raster(folder.model.schema_1d2d_ggg.rasters.dem.path)

# Set Dem path if its not found
# dem_path_dropdown = widgets.Dropdown(
#     options="",
#     disabled=False,
#     layout=item_layout(grid_area="dem_path_dropdown"),
# )


# if folder.model.schema_base.rasters.find_dem() == "":
#     dem_path_dropdown.options = [
#         i.split(os.sep)[-1] for i in folder.model.schema_base.rasters.find_ext("tif")
#     ]
# else:
#     dem_path_dropdown.options = [folder.model.schema_base.rasters.dem.name]
# # Batch folder selection


# Display precipitation zones
polder_shape = gpd.read_file(folder.source_data.polder_polygon.path)
with pkg_resources.path(htt.resources, "precipitation_zones_hhnk.tif") as p:
    precip_zones_raster = hrt.Raster(p.absolute().as_posix())
    neerslag_array = precip_zones_raster.get_array(band_count=3)

with pkg_resources.path(htt.resources, "precipitation_frequency.xlsx") as p:
    freqs = pd.read_excel(p.absolute().as_posix(), engine="openpyxl")

fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(neerslag_array, extent=precip_zones_raster.metadata.bounds)
polder_shape.plot(ax=ax, color='red')


precipitation_zone_box = widgets.Select(
    options=["hevig", "debilt"],
    rows=2,
    disabled=False,
    value=None,
    layout=item_layout(grid_area="precipitation_zone_box"),
)

print("Selecteer map met batch resultaten")
display(output_folder_box)

print("Selecteer neerslagzone")
display(precipitation_zone_box)
print("Geselecteerd DEM bestand")
print(dem)
# display(dem_path_dropdown)

## Lokaliseren polder folder

In [ ]:
if output_folder_box.value == "":
    raise Exception("Select batch folder")

batch_fd = folder.threedi_results.batch[output_folder_box.value]


if dem.metadata.pixel_width != 0.5:
    new_dem_path = batch_fd.downloads.pl/f"{dem.pl.stem}_05m.tif"
    dem = hrt.Raster(new_dem_path)


df = pd.DataFrame(batch_fd.downloads.names, columns=["dl_name"])
for dl_name in batch_fd.downloads.names:
    df.loc[df["dl_name"] == dl_name, "depth_max"] = getattr(
        batch_fd.downloads, dl_name
    ).depth_max.path
    df.loc[df["dl_name"] == dl_name, "damage_total"] = getattr(
        batch_fd.downloads, dl_name
    ).damage_total.path


## %%

freqs = freqs[["dl_name", "freq_{}_jaar".format(precipitation_zone_box.value)]]
freqs.rename(
    {"freq_{}_jaar".format(precipitation_zone_box.value): "freq_jaar"},
    axis=1,
    inplace=True,
)

df_freqs = df.merge(freqs, on="dl_name")

## %% Aanmaken of laden peilgebieden polygon
if not folder.source_data.peilgebieden.peilgebieden.exists:
    fixeddrainage = folder.source_data.datachecker.load("fixeddrainagelevelarea")
    fixeddrainage.to_file(folder.source_data.peilgebieden.peilgebieden.path)
    print(
        f"Peilgebieden shapefile aangemaakt: {folder.source_data.peilgebieden.peilgebieden.name}.shp"
    )
else:
    print(
        f"Peilgebieden shapefile gevonden: {folder.source_data.peilgebieden.peilgebieden.name}.shp"
    )

## Input klaarzetten

In [ ]:
klimaatsommenrep = KlimaatsommenPrep(folder=folder,
    batch_name=batch_fd.name,
    cfg_file = 'cfg_lizard.cfg',
    landuse_file = r"\\corp.hhnk.nl\data\Hydrologen_data\Data\01.basisgegevens\rasters\landgebruik\landuse2019_tiles\combined_rasters.vrt",
    verify=True
)

klimaatsommenrep.run(gridgpkg=True, 
                    depth=True, 
                    dmg=True, 
                    wlvl=False, 
                    overwrite=False)

## Maskerkaart aanmaken

In [ ]:
# Aanmaken polygon van maskerkaart
maskerkaart.command(
    path_piek=batch_fd.downloads.piek_ghg_T1000.netcdf.path,
    path_blok=batch_fd.downloads.blok_ghg_T1000.netcdf.path,
    path_out=batch_fd.output.maskerkaart.path,
)

# Omzetten polygon in raster voor diepteraster
mask_depth = rasterize_maskerkaart(
    input_file=batch_fd.output.maskerkaart.path,
    mask_plas_path=batch_fd.output.mask_diepte_plas.path,
    mask_overlast_path=batch_fd.output.mask_diepte_overlast.path,
    meta=hrt.Raster(batch_fd.downloads.piek_glg_T10.depth_max.path).metadata,
)

# Omzetten polygon in raster voor schaderaster (kan verschillen van diepte met andere resolutie)
mask_damage = rasterize_maskerkaart(
    input_file=batch_fd.output.maskerkaart.path,
    mask_plas_path=batch_fd.output.mask_schade_plas.path,
    mask_overlast_path=batch_fd.output.mask_schade_overlast.path,
    meta=hrt.Raster(batch_fd.downloads.piek_glg_T10.damage_total.path).metadata,
)

## Peilgebieden rasterizeren

In [ ]:
for raster_type, raster_name in zip(
    ["depth_max", "damage_total"], ["diepte", "schade"]
):
    peilgebieden.rasterize_peilgebieden(
        input_raster=hrt.Raster(df.iloc[0][raster_type]),
        output_file=getattr(batch_fd.output.temp, f"peilgebieden_{raster_name}"),
        input_peilgebieden=folder.source_data.peilgebieden.peilgebieden,
        output_peilgebieden=batch_fd.output.temp.peilgebieden,
        mask_path=batch_fd.output.maskerkaart.path,
        overwrite=False,
    )

## Ruimtekaart aanmaken

In [ ]:
if not batch_fd.output.ruimtekaart.exists:
    # if True:
    ruimtekaart.create_ruimtekaart(
        pgb_path=batch_fd.output.temp.peilgebieden.path,
        output_path=batch_fd.output.ruimtekaart.path,
        batch_fd=batch_fd,
    )
    print("Ruimtekaart created")

## Interpoleren van diepterasters

In [ ]:
diepte_rasters = df_freqs["depth_max"].values
frequenties = df_freqs["freq_jaar"].values


for T in [10, 100, 1000]:

    output_file = getattr(batch_fd.output, f"depth_T{T}_totaal")

    # Voor de gegeven herhalingstijd, interpoleer de rasters.
    main_interpolate_rasters(
        T=T,
        output_file=output_file,
        rasters=diepte_rasters,
        frequenties=frequenties,
        output_nodata=-9999.00,
        dem_path=dem.source_path,
        extra_nodata_value=0,
    )

#### Plas en overlast kaarten maken

In [ ]:
for T in [10, 100, 1000]:
    input_raster = hrt.Raster(getattr(batch_fd.output, f"depth_T{T}_totaal").path)

    # Creeer masker voor plas en overlast
    for mask_type in ["plas", "overlast"]:
        mask = hrt.Raster(getattr(batch_fd.output, f"mask_diepte_{mask_type}").path)

        output_raster = getattr(batch_fd.output, f"depth_T{T}_{mask_type}")

        if not output_raster.exists:
            raster_array = input_raster._read_array()
            mask_array = mask._read_array()

            raster_array[~(mask_array == 1)] = input_raster.nodata  # Maskeer
            hrt.save_raster_array_to_tiff(
                output_file=output_raster.path,
                raster_array=raster_array,
                nodata=input_raster.nodata,
                metadata=input_raster.metadata,
            )
            print(f"{output_raster.name} created")

        else:
            print(f"{output_raster.name} already exists")
raster_array = None
mask_array = None

## Schadekaart maken

In [ ]:
dv = 0.04  # discontovoet [%]
n = 50  # investeringstermijn [jaren]
schade_rasters = df_freqs["damage_total"].values
frequencies = df_freqs["freq_jaar"].values
output_raster = batch_fd.output.cw_schade_totaal

if not output_raster.exists:
    schadekaart.main_maak_schadekaart(
        output_raster=output_raster,
        schade_rasters=schade_rasters,
        frequencies=frequencies,
        output_nodata=0,
        dv=dv,
        n=n,
    )

    print(f"{output_raster.name} created")
else:
    print(f"{output_raster.name} already exists")

#### Maak masker voor plas en overlast

In [ ]:
input_raster = hrt.Raster(batch_fd.output.cw_schade_totaal.path)

# Creeer masker voor plas en overlast
for mask_type in ["plas", "overlast"]:
    mask = hrt.Raster(getattr(batch_fd.output, f"mask_schade_{mask_type}").path)

    output_raster = getattr(batch_fd.output, f"cw_schade_{mask_type}")

    if not output_raster.exists:
        raster_array = input_raster._read_array()
        mask_array = mask._read_array()

        raster_array[~(mask_array == 1)] = input_raster.nodata
        hrt.save_raster_array_to_tiff(
            output_file=output_raster.path,
            raster_array=raster_array,
            nodata=input_raster.nodata,
            metadata=input_raster.metadata,
        )
        print(f"{output_raster.name} created")

    else:
        print(f"{output_raster.name} already exists")
raster_array = None
mask_array = None

## Bereken schade per gebied

In [ ]:
schade_gdf = gpd.read_file(batch_fd.output.temp.peilgebieden.path)
labels_raster = hrt.Raster(batch_fd.output.temp.peilgebieden_schade.path)
labels_index = schade_gdf["index"].values

output_file = batch_fd.output.schade_peilgebied.path


# Bereken totale schade per peilgebied voor de twee gemaskerde schaderasters.
for mask_type, mask_name in zip(["plas", "overlast"], ["mv", "ws"]):
    schade_raster = hrt.Raster(getattr(batch_fd.output, f"cw_schade_{mask_type}").path)

    # Calculate sum per region
    accum = schade_raster.sum_labels(
        labels_raster=labels_raster, labels_index=labels_index
    )

    schade_gdf[f"cw_{mask_name}"] = accum

schade_gdf["cw_tot"] = schade_gdf["cw_ws"] + schade_gdf["cw_mv"]

# schade_gdf = schade_gdf.loc[schade_gdf['cw_tot'] > 0.0]


schade_per_polder = (
    schade_gdf[["name", "cw_tot", "cw_ws", "cw_mv"]]
    .groupby("name")
    .sum()
    .sort_values(by="cw_ws", ascending=False)
)

# Opslaan naar shapefile en csv
schade_gdf.to_file(output_file)
schade_per_polder.to_csv(batch_fd.output.schade_polder.path)

In [ ]:
schade_gdf

# Schade corrigeren
Als er schade op treedt op plekken waar dit niet logisch is, kan je een shapefile aanmaken die we vervolgens uit het raster zullen knippen. Maak hiervoor het volgende bestand aan:
.\01. DAMO HDB en Datachecker\peilgebieden\geen_schade.shp

In [ ]:
raster = hrt.Raster(str(batch_fd.output.cw_schade_plas))

dv = 0.04 #discontovoet [%]
n = 50 #investeringstermijn [jaren]
cw_factor = (1 - (1 - dv) ** n) / dv
pixel_factor = raster.metadata['pixel_width']**2/0.25 #niet nodig als resolutie goed staat


## Verwijder onrealistische schades


In [ ]:
maskerkaart2 = gpd.read_file(str(folder.source_data.peilgebieden.geen_schade)) #load maskerkaart (geen_schade.shp)

maskerkaart_union = maskerkaart2.buffer(0.1).unary_union.buffer(-0.1) #make one geometry from gdf.
    
#Rasterize polygon
maskerkaart_union = gpd.GeoDataFrame(geometry=[maskerkaart_union])
#Voeg kolom toe aan gdf, deze waarden worden in het raster gezet.
maskerkaart_union['val']=1

mask = hrt.gdf_to_raster(maskerkaart_union, value_field='val', raster_out='', nodata=0, metadata=damage_meta, epsg=28992, driver='MEM')
mask = mask > 0

In [ ]:
for mask_type, mask_name in zip(["plas", "overlast"], ["mv", "ws"]):
    schade_raster = hrt.Raster(getattr(batch_fd.output, f"cw_schade_{mask_type}").path)
    output = str(batch_fd.output) +  f"/cw_schade_{mask_type}_correctie.tif"
    array = schade_raster.get_array()
    array[mask] = raster.nodata
    hrt.save_raster_array_to_tiff(output, array, raster.nodata, raster.metadata)

In [ ]:
schade_raster_corr_file={"plas": str(batch_fd.output.cw_schade_plas_corr),
                        "overlast": str(batch_fd.output.cw_schade_overlast_corr)}
schades, schade_per_polder = maak_schade_polygon(peilgebiedenbestand=str(folder.source_data.peilgebieden.peilgebieden), 
                                                 schade_raster_file=schade_raster_corr_file, 
                                                 pixel_factor=pixel_factor, 
                                                 output_schade_file=str(batch_fd.output.schade_peilgebied_corr), 
                                                 output_polder_file=str(batch_fd.output.schade_polder_corr))



In [ ]:
dir(htt.core.climate_scenarios)

In [ ]:
import importlib
importlib.reload(maak_schade_polygon)